In [ ]:
using BenchmarkTools

In [ ]:
include("/home/kwat/github/Kraft.jl/src/Kraft.jl")

In [ ]:
const genes = readlines("/home/kwat/github/kraft/notebook/genes.txt")[2:end]

In [ ]:
const scores = fill(
    1 / length(genes),
    length(genes),
)

In [ ]:
gmt_file_path = "/home/kwat/garden/data/gene_set/msigdb_v6.2/h.all.v6.2.symbols.gmt"
# gmt_file_path = "/home/kwat/garden/data/gene_set/msigdb_v6.2/c3.all.v6.2.symbols.gmt"

const gene_set_dict = Kraft.read_gmt(gmt_file_path)

In [ ]:
const gene_set_genes = gene_set_dict[sort(collect(keys(gene_set_dict)))[1]]

## Benchmark Julia

In [ ]:
@benchmark Kraft.compute_gene_set_enrichment(
    scores,
    genes,
    gene_set_genes,
)

In [ ]:
benchmark_result = @benchmark Kraft.compute_gene_set_enrichment(
    scores,
    genes,
    gene_set_dict,
)

ms_per_gene_set = round(
    minimum(benchmark_result.times) / 1e6 / length(gene_set_dict),
    sigdigits=2,
)

println("$ms_per_gene_set ms / gene set")

benchmark_result

## Benchmark Python

In [ ]:
using PyCall

In [ ]:
pd = pyimport("pandas")

In [ ]:
kraft = pyimport("kraft")

In [ ]:
const gene_score = pd.Series(
    scores,
    index=genes,
)

@benchmark kraft.run_single_sample_gsea(
    gene_score,
    gene_set_genes,
    plot=false,
)

In [ ]:
const gene_x_sample = gene_score.to_frame()

const gene_sets = kraft.read_gmt(gmt_file_path)

benchmark_result = @benchmark kraft.run_single_sample_gseas(
    gene_x_sample,
    gene_sets,
)

ms_per_gene_set = round(
    minimum(benchmark_result.times) / 1e6 / length(gene_set_dict),
    sigdigits=2,
)

println("$ms_per_gene_set ms / gene set")

benchmark_result

## Gene x Sample ==> Gene Set x Sample

In [2]:
using TableReader

In [ ]:
gene_x_sample = readdlm("/home/kwat/github/match_pattern/output/gene_x_sample.processed.tsv")

In [4]:
for  

,Gene,WC1,WC2,WC3,WM1,WM2,WM3
,String,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,A1BG,0.0664477,0.038043,0.12041,0.0919792,missing,0.0630237
2,A1CF,0.00790968,missing,0.0134654,0.00591755,missing,0.00909115
3,A2M,missing,missing,missing,missing,0.166227,0.0118349
4,A2ML1,0.106133,0.0852753,0.0841,0.145724,0.0769687,0.175137
5,A2MP1,missing,0.016517,missing,missing,missing,missing
6,A3GALT2,0.264308,0.165927,missing,0.135154,missing,missing
7,A4GALT,0.681983,0.916425,0.737331,0.2952,0.347736,0.322686
8,AAAS,0.449162,0.867066,0.494286,0.620642,1.49168,0.717134
9,AACS,0.161937,0.292978,0.212836,0.302402,0.206749,0.42944
